### Setup
Imports core libs (async, I/O, dates) plus `cognee`, `praw` (Reddit), and `feedparser`.  
Loads secrets from `.env` to configure APIs and runtime.


In [1]:
# 🔐 Environment Variables Setup

# TAVILY_API_KEY=your_tavily_api_key_here
# LLM_API_KEY=your_llm_api_key_here
# REDDIT_CLIENT_ID=your_reddit_client_id_here
# REDDIT_CLIENT_SECRET=your_reddit_client_secret_here
# REDDIT_USER_AGENT=your_app_name_or_username

In [2]:
# Import all necessary libraries
import asyncio
import os
import cognee
import praw
import tempfile
import feedparser
from openai import OpenAI
from openai import AsyncOpenAI
import os

from datetime import datetime, timedelta
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables from .env file
load_dotenv()


2025-11-04T10:07:10.600631 [info     ] Deleted old log file: /Users/lstromann/projects/blogs/stay_on_top_of_news/.venv/lib/python3.11/site-packages/logs/2025-10-30_17-44-33.log [cognee.shared.logging_utils]

2025-11-04T10:07:10.977474 [info     ] Logging initialized            [cognee.shared.logging_utils] cognee_version=0.3.7 database_path=/Users/lstromann/projects/blogs/stay_on_top_of_news/.venv/lib/python3.11/site-packages/cognee/.cognee_system/databases graph_database_name= os_info='Darwin 24.6.0 (Darwin Kernel Version 24.6.0: Mon Jul 14 11:30:40 PDT 2025; root:xnu-11417.140.69~1/RELEASE_ARM64_T8132)' python_version=3.11.13 relational_config=cognee_db structlog_version=25.5.0 vector_config=lancedb

2025-11-04T10:07:10.977865 [info     ] Database storage: /Users/lstromann/projects/blogs/stay_on_top_of_news/.venv/lib/python3.11/site-packages/cognee/.cognee_system/databases [cognee.shared.logging_utils]


True

### Clean Slate
Wipes previously stored documents, nodes, and relationships. Only use this when you want a fresh/ cleaned memory
Clears system-level caches/metadata (indexes, config history) to ensure a fresh run.


In [3]:
if True:
    # delete stored data (documents, nodes, relationships)
    await cognee.prune.prune_data()
    print("Data pruned.")

    # clear system-level caches and metadata (indexes, config history, etc.)
    await cognee.prune.prune_system(metadata=True)
    print("Metadata pruned.")

Data pruned.



2025-11-04T10:07:11.772042 [info     ] Loaded JSON extension          [cognee.shared.logging_utils]

2025-11-04T10:07:11.840284 [info     ] Deleted Kuzu database files at /Users/lstromann/projects/blogs/stay_on_top_of_news/.venv/lib/python3.11/site-packages/cognee/.cognee_system/databases/cognee_graph_kuzu [cognee.shared.logging_utils]

2025-11-04T10:07:14.475365 [info     ] Database deleted successfully. [cognee.shared.logging_utils]

Storage manager absolute path: /Users/lstromann/projects/blogs/stay_on_top_of_news/.venv/lib/python3.11/site-packages/cognee/.cognee_cache

Deleting cache...             

✓ Cache deleted successfully! 


Metadata pruned.


## Initialize Cognee & Define Catchiness Functions
After pruning, we need to reinitialize cognee's infrastructure before using LLM functions.

### Ingest Reddit
Initializes Reddit API via PRAW using env credentials.  
Fetches latest posts from selected subreddits, writes them to temp files.  
Adds those files to Cognee for ingestion, then removes the temp files.


In [ ]:
if True:
    #########################################################
    # Add reddit data to cognee pipeline
    #########################################################

    # --- Initialize Reddit API ---
    reddit = praw.Reddit(
        client_id=os.getenv("REDDIT_CLIENT_ID"),
        client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
        user_agent=os.getenv("REDDIT_USER_AGENT", "stay_on_top_of_news/1.0")
    )

    print("Reddit API initialized.")


    async def get_reddit_posts(subreddit_name, limit=25):
        """Fetch recent posts from a subreddit"""
        posts_text = []
        subreddit = reddit.subreddit(subreddit_name)
        
        for post in subreddit.new(limit=limit):
            # Evaluate catchiness using LLM (3 lines)            
            content = post.selftext if post.selftext else '[Link post - no text content]'
            prompt = f"Rate this news catchiness (1-10). Catchiness = controversial + attention-capturing + timeless + major breakthrough.\n\nTitle: {post.title}\nSummary: {content[:500]}\n\nReturn ONLY integer 1-10."
            
            client = AsyncOpenAI(api_key=os.getenv("LLM_API_KEY"), timeout=5.0)
            resp = await client.chat.completions.create(model="gpt-4o-mini", messages=[{"role": "user", "content": prompt}], max_tokens=5)
            score = int(resp.choices[0].message.content.strip())
                        
            post_content = f"""
                            Title: {post.title}
                            Subreddit: r/{subreddit_name}
                            Author: u/{post.author.name if post.author else '[deleted]'}
                            Score: {post.score}
                            Created: {datetime.fromtimestamp(post.created_utc)}
                            URL: {post.url}
                            Catchiness Score: {score}/10

                            Content:
                            {content}

                            ---
                            """
            posts_text.append(post_content)







        return "\n".join(posts_text)


    # Create temporary files for each subreddit's posts and define subreddits
    subreddits = [
        # "MachineLearning",
        "OpenAI",
        # "artificial",
        # "LocalLLaMA",
        # "AIMemory",
        # "GraphRAG",
        # "ollama",
        # "LLMDevs",
        # "Rag",
        # "datascience",
        # "ai",
        # "ArtificialInteligence", 
        # "deepLearning",
        # "AI_Agents",
        # "ChatGPT",
        # "Singularity",
        # "StableDiffusion",
        # "Midjourney",
        # "generative",
        # "aipromptprogramming",
        # "aiart"
    ]
    temp_files = []

    for sub in subreddits:
        try:
            print(f"Fetching posts from r/{sub}...")
            posts_content = await get_reddit_posts(sub, limit=25)
            
            # Create a temporary file with the posts
            temp_file = tempfile.NamedTemporaryFile(
                mode='w', 
                suffix=f'_{sub}_posts.txt',
                delete=False,
                encoding='utf-8'
            )
            temp_file.write(posts_content)
            temp_file.close()
            temp_files.append(temp_file.name)
            print(f"  ✓ Fetched and saved posts from r/{sub}")
        except Exception as e:
            print(f"  ⚠️  Error fetching r/{sub}: {type(e).__name__} - {str(e)}")
            print(f"  → Skipping subreddit and continuing...")


    # --- Add Reddit posts to cognee ---
    await cognee.add(temp_files)
    print("✓ Reddit posts added to cognee")

    # Clean up temporary files
    for temp_file in temp_files:
        try:
            Path(temp_file).unlink()
        except:
            pass


Reddit API initialized.
Fetching posts from r/OpenAI...
  ✓ Fetched and saved posts from r/OpenAI
Fetching posts from r/LocalLLaMA...
  ✓ Fetched and saved posts from r/LocalLLaMA
Fetching posts from r/AIMemory...
  ✓ Fetched and saved posts from r/AIMemory
Fetching posts from r/ollama...
  ✓ Fetched and saved posts from r/ollama
Fetching posts from r/LLMDevs...
  ✓ Fetched and saved posts from r/LLMDevs
Fetching posts from r/Rag...
  ✓ Fetched and saved posts from r/Rag
Fetching posts from r/datascience...
  ✓ Fetched and saved posts from r/datascience
Fetching posts from r/ArtificialInteligence...
  ✓ Fetched and saved posts from r/ArtificialInteligence
Fetching posts from r/deepLearning...
  ✓ Fetched and saved posts from r/deepLearning
Fetching posts from r/AI_Agents...
  ✓ Fetched and saved posts from r/AI_Agents
Fetching posts from r/ChatGPT...
  ✓ Fetched and saved posts from r/ChatGPT
User 0a9b8f79-660c-46f3-8854-ee5e64648d87 has registered.



2025-11-04T10:10:38.335886 [info     ] Pipeline run started: `07bceca6-6bc6-5dff-9a70-8258990689eb` [run_tasks_with_telemetry()]

2025-11-04T10:10:38.513808 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]

2025-11-04T10:10:38.695522 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]

2025-11-04T10:10:38.882542 [info     ] Pipeline run started: `07bceca6-6bc6-5dff-9a70-8258990689eb` [run_tasks_with_telemetry()]

2025-11-04T10:10:39.062616 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]

2025-11-04T10:10:39.242515 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]

2025-11-04T10:10:39.420591 [info     ] Pipeline run started: `07bceca6-6bc6-5dff-9a70-8258990689eb` [run_tasks_with_telemetry()]

2025-11-04T10:10:39.660831 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]

2025-11-04T10:10:39.834808 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]



✓ Reddit posts added to cognee


### Ingest RSS News
Defines a small set of AI/tech RSS feeds.  
Parses each feed, formats the latest entries, saves to temp text files.  
Adds all parsed feeds to Cognee and cleans up the temp files.


In [ ]:

#########################################################
# News Sources
#########################################################

# minimal rss list you can extend
rss_feeds = [
	# --- Research ---
	"https://export.arxiv.org/rss/cs.LG",           # machine learning
	# "https://export.arxiv.org/rss/cs.AI",           # artificial intelligence
	# "https://export.arxiv.org/rss/cs.CL",           # computation & language
	# "https://export.arxiv.org/rss/cs.CV",           # computer vision
	# "https://export.arxiv.org/rss/stat.ML",         # statistical ML
	# "https://export.arxiv.org/rss/eess.IV",         # image/video systems (vision)

	# --- Major AI labs & research orgs ---
	"https://openai.com/news/rss.xml",
	# "https://ai.googleblog.com/feeds/posts/default?alt=rss",
	# "https://deepmind.google/discover/blog/feed.xml",     # DeepMind
	# "https://research.ibm.com/blog/rss.xml",              # IBM Research
	# "https://blogs.microsoft.com/ai/feed/",               # Microsoft AI blog
	# "https://ai.meta.com/blog/rss.xml",                   # Meta AI
	# "https://huggingface.co/blog/feed.xml",               # Hugging Face

	# --- Industry / analysis / newsletters ---
	# "https://www.technologyreview.com/feed/",             # MIT Tech Review
	# "https://www.therundown.ai/feed",                     # The Rundown AI
	# "https://www.theneurondaily.com/feed",                # The Neuron
	# "https://www.semiaanalysis.com/feed",                 # SemiAnalysis
	# "https://www.oneusefulthing.org/feed",                # Ethan Mollick
	# "https://importai.substack.com/feed",                 # Jack Clark’s Import AI
	# "https://www.lesswrong.com/feed.xml",                 # AI safety / rationalist posts
	# "https://alignmentforum.org/feed.xml",                # alignment research forum

	# --- Tech / product news outlets ---
	# "https://techcrunch.com/tag/ai/feed/",
	# "https://www.theverge.com/rss/index.xml",
	# "https://venturebeat.com/category/ai/feed/",
	# "https://www.wired.com/feed/category/ai/latest/rss",  # Wired AI
	# "https://thenextweb.com/feed/",                       # The Next Web
	# "https://spectrum.ieee.org/feed",                     # IEEE Spectrum (engineering AI)

	# --- Policy, ethics, society ---
	# "https://hai.stanford.edu/rss.xml",                   # Stanford HAI
	# "https://www.brookings.edu/topic/artificial-intelligence/feed/",  # Brookings AI policy
	# "https://datainnovation.org/feed/",                   # Center for Data Innovation

	# --- Optional fun / creative ---
	# "https://towardsdatascience.com/feed",                # Medium ML articles
	# "https://www.kdnuggets.com/feed",                     # data science & AI
]


# Parse RSS feeds and save to temp files
print(f"\nParsing {len(rss_feeds)} RSS feeds...")
rss_temp_files = []

for feed_url in rss_feeds:
    try:
        print(f"  Parsing {feed_url}...")
        feed = feedparser.parse(feed_url)
        
        if not feed.entries:
            print(f"  ⚠️  No entries found")
            continue
        
        # Format feed content
        feed_text = [f"RSS Feed: {feed.feed.get('title', feed_url)}\n"]
        feed_text.append("=" * 80 + "\n")
        
        for entry in feed.entries[:10]:  # Get latest 10 entries
            # Evaluate catchiness using LLM
            title, summary = entry.get('title', 'No title'), entry.get('summary', entry.get('description', 'No summary available'))
            prompt = f"Rate this news catchiness (1-10). Catchiness = controversial + attention-capturing + timeless + major breakthrough.\n\nTitle: {title}\nSummary: {summary[:500]}\n\nReturn ONLY integer 1-10."
            
            client = AsyncOpenAI(api_key=os.getenv("LLM_API_KEY"), timeout=5.0)
            resp = await client.chat.completions.create(model="gpt-4o-mini", messages=[{"role": "user", "content": prompt}], max_tokens=5)
            score = int(resp.choices[0].message.content.strip())
            
            
            feed_text.append(f"""
                Title: {entry.get('title', 'No title')}
                Published: {entry.get('published', entry.get('updated', 'N/A'))}
                Link: {entry.get('link', 'N/A')}
                Catchiness Score: {score}/10

                Summary:
                {entry.get('summary', entry.get('description', 'No summary available'))}

                ---
                """)
        
        # Save to temp file
        temp_file = tempfile.NamedTemporaryFile(
            mode='w', suffix='_rss.txt', delete=False, encoding='utf-8'
        )
        temp_file.write("\n".join(feed_text))
        temp_file.close()
        rss_temp_files.append(temp_file.name)
        print(f"  ✓ Saved")
        
    except Exception as e:
        print(f"  ✗ Failed: {e}")

# Add RSS feeds to cognee
if rss_temp_files:
    print("\nAdding RSS feeds to cognee...")
    await cognee.add(rss_temp_files)
    print("✓ RSS feeds added")
    
    # Cleanup
    for temp_file in rss_temp_files:
        try:
            Path(temp_file).unlink()
        except:
            pass
else:
    print("⚠️  No RSS feeds were parsed")




Parsing 19 RSS feeds...
  Parsing https://export.arxiv.org/rss/cs.LG...
  ✓ Saved
  Parsing https://export.arxiv.org/rss/cs.AI...
  ✓ Saved
  Parsing https://export.arxiv.org/rss/cs.CL...
  ✓ Saved
  Parsing https://export.arxiv.org/rss/cs.CV...
  ✓ Saved
  Parsing https://export.arxiv.org/rss/stat.ML...
  ✓ Saved
  Parsing https://openai.com/news/rss.xml...
  ✓ Saved
  Parsing https://deepmind.google/discover/blog/feed.xml...
  ⚠️  No entries found
  Parsing https://research.ibm.com/blog/rss.xml...
  ⚠️  No entries found
  Parsing https://blogs.microsoft.com/ai/feed/...
  ✓ Saved
  Parsing https://ai.meta.com/blog/rss.xml...
  ⚠️  No entries found
  Parsing https://www.technologyreview.com/feed/...
  ✓ Saved
  Parsing https://www.therundown.ai/feed...
  ⚠️  No entries found
  Parsing https://www.theneurondaily.com/feed...
  ⚠️  No entries found
  Parsing https://www.semiaanalysis.com/feed...
  ⚠️  No entries found
  Parsing https://www.oneusefulthing.org/feed...
  ✓ Saved
  Parsing h


2025-11-04T10:12:23.420609 [info     ] Pipeline run started: `07bceca6-6bc6-5dff-9a70-8258990689eb` [run_tasks_with_telemetry()]


  ⚠️  No entries found

Adding RSS feeds to cognee...



2025-11-04T10:12:23.600144 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]

2025-11-04T10:12:23.786700 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]

2025-11-04T10:12:23.973809 [info     ] Pipeline run started: `07bceca6-6bc6-5dff-9a70-8258990689eb` [run_tasks_with_telemetry()]

2025-11-04T10:12:24.188213 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]

2025-11-04T10:12:24.460587 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]

2025-11-04T10:12:24.637826 [info     ] Pipeline run started: `07bceca6-6bc6-5dff-9a70-8258990689eb` [run_tasks_with_telemetry()]

2025-11-04T10:12:24.814812 [info     ] Coroutine task started: `resolve_data_directories` [run_tasks_base]

2025-11-04T10:12:24.994063 [info     ] Coroutine task started: `ingest_data` [run_tasks_base]

2025-11-04T10:12:25.208132 [info     ] Pipeline run started: `07bceca6-6bc6-5dff-9a70-8258990689eb` [run_tasks_with_telemetry()]



✓ RSS feeds added


### Build & Enhance the Graph
`cognify()` extracts entities/relations to build the knowledge graph.  
Saves an initial HTML visualization.  
`memify()` consolidates memory to strengthen connections.  
Saves an enhanced HTML visualization for comparison.


In [6]:
# extract knowledge from the scraped data
await cognee.cognify()
print("Knowledge graph created.")

# visualize the knowledge graph
simple_graph_visualization_path = str(Path.cwd() / "simple_graph_visualization.html")
await cognee.visualize_graph(simple_graph_visualization_path)
print(f"Data visualized")

# enhance the knowledge graph with memory consolidation for improved connections
await cognee.memify()
print("Knowledge graph enhanced with memory consolidation.")

# generate the second graph visualization after memory enhancement
enhanced_graph_visualization_path = str(Path.cwd() / "enhanced_graph_visualization.html")
await cognee.visualize_graph(enhanced_graph_visualization_path)
print(f"Enhanced graph visualized: {enhanced_graph_visualization_path}")



2025-11-04T10:12:37.351050 [info     ] Ontology file 'None' not found. No owl ontology will be attached to the graph. [OntologyAdapter]

2025-11-04T10:12:37.380988 [info     ] Pipeline run started: `00001b5f-9b81-527a-a9b3-2db4c99498d6` [run_tasks_with_telemetry()]

2025-11-04T10:12:37.551674 [info     ] Coroutine task started: `classify_documents` [run_tasks_base]

2025-11-04T10:12:37.727914 [info     ] Coroutine task started: `check_permissions_on_dataset` [run_tasks_base]

2025-11-04T10:12:37.909601 [info     ] Pipeline run started: `00001b5f-9b81-527a-a9b3-2db4c99498d6` [run_tasks_with_telemetry()]

2025-11-04T10:12:38.110648 [info     ] Coroutine task started: `classify_documents` [run_tasks_base]

2025-11-04T10:12:38.284393 [info     ] Coroutine task started: `check_permissions_on_dataset` [run_tasks_base]

2025-11-04T10:12:38.470340 [info     ] Pipeline run started: `00001b5f-9b81-527a-a9b3-2db4c99498d6` [run_tasks_with_telemetry()]

2025-11-04T10:12:38.646486 [info     ] Corou

Knowledge graph created.



2025-11-04T10:18:25.849581 [info     ] Graph visualization saved as /Users/lstromann/projects/blogs/stay_on_top_of_news/simple_graph_visualization.html [cognee.shared.logging_utils]

2025-11-04T10:18:25.850844 [info     ] The HTML file has been stored at path: /Users/lstromann/projects/blogs/stay_on_top_of_news/simple_graph_visualization.html [cognee.shared.logging_utils]

2025-11-04T10:18:25.906831 [info     ] Graph projection completed: 1879 nodes, 5671 edges in 0.05s [CogneeGraph]

2025-11-04T10:18:25.918614 [info     ] Pipeline run started: `c25e5c09-bd36-5606-b721-e872430a823a` [run_tasks_with_telemetry()]


Data visualized



2025-11-04T10:18:26.088883 [info     ] Async Generator task started: `extract_subgraph_chunks` [run_tasks_base]

2025-11-04T10:18:26.260219 [info     ] Coroutine task started: `add_rule_associations` [run_tasks_base]

2025-11-04T10:19:33.774971 [info     ] Coroutine task completed: `add_rule_associations` [run_tasks_base]

2025-11-04T10:19:33.950620 [info     ] Coroutine task started: `add_rule_associations` [run_tasks_base]

2025-11-04T10:20:09.454843 [info     ] Coroutine task completed: `add_rule_associations` [run_tasks_base]

2025-11-04T10:20:09.635707 [info     ] Coroutine task started: `add_rule_associations` [run_tasks_base]

2025-11-04T10:21:50.771170 [info     ] Coroutine task completed: `add_rule_associations` [run_tasks_base]

2025-11-04T10:21:50.958883 [info     ] Coroutine task started: `add_rule_associations` [run_tasks_base]

2025-11-04T10:23:03.505569 [info     ] Coroutine task completed: `add_rule_associations` [run_tasks_base]

2025-11-04T10:23:03.687341 [info     ]

Knowledge graph enhanced with memory consolidation.
Enhanced graph visualized: /Users/lstromann/projects/blogs/stay_on_top_of_news/enhanced_graph_visualization.html


### Cross-Document Query
Runs a GRAPH_COMPLETION search: *“What were the main events in AI this week?”*  
Returns a synthesized answer aggregated across Reddit + RSS sources.


In [7]:
# demonstrate cross-document knowledge retrieval from multiple data sources
results = await cognee.search(
    query_text="What were the main events in the AI world this week?",
    query_type=cognee.SearchType.GRAPH_COMPLETION,
)
print(f"Search results: {results[0]}")


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x13c0392d0>

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x13bf7dfd0>

2025-11-04T12:08:28.654206 [info     ] Graph projection completed: 2076 nodes, 6069 edges in 0.25s [CogneeGraph]

2025-11-04T12:08:29.789714 [info     ] Vector collection retrieval completed: Retrieved distances from 6 collections in 0.89s [cognee.shared.logging_utils]


Search results: Brief weekly roundup (from provided context):

- Big Reddit conversation about “What’s actually going on with AI?” — users debating whether current investment and progress are hype or truly transformative. 
- Mercor AI / Forbes story (paper‑billionaire founders) sparked skepticism and threads about startup valuations and dilution. 
- OpenAI news and disruption: a major promotion — 12 months of ChatGPT Go free in India — plus user reports of access/payment/subscription problems and missing UI features (Deep Research with GPT‑5). 
- New research/claim: an arXiv paper showing LLMs can “talk to each other without using words” circulated in r/OpenAI. 
- Safety/behavior incidents and demos: Grok/agent (“Bad Rudi”) reportedly able to bypass Alexa protections and place orders; users sharing surprising consumer AI outputs (e.g., realistic “AI baby” generators). 
- Ongoing enterprise discussion about the AI trust gap — calls for explainability, oversight, and accountability as ba

# Stuff

In [8]:
# generate the second graph visualization after memory enhancement using custom visual.py
import importlib
import visual
importlib.reload(visual)  # Reload to pick up latest changes
from visual import cognee_network_visualization
from cognee.infrastructure.databases.graph import get_graph_engine

# Get graph data from cognee
graph_engine = await get_graph_engine()
graph_data = await graph_engine.get_graph_data()

# Use custom visualization with catchiness-based brightness
fancy_graph_visualization_path = str(Path.cwd() / "fancy_graph_visualization_v2.html")
await cognee_network_visualization(graph_data, fancy_graph_visualization_path)
print(f"Fancy graph visualized with custom catchiness coloring: {fancy_graph_visualization_path}")


2025-11-04T12:08:41.543050 [info     ] Graph visualization saved as /Users/lstromann/projects/blogs/stay_on_top_of_news/fancy_graph_visualization_v2.html [cognee.shared.logging_utils]



=== DocumentChunk #1 Debug ===
Text preview (first 200 chars): RSS Feed: MIT Technology Review



                Title: The State of AI: Is China about to win the race?
            
Catchiness match: <re.Match object; span=(385, 407), match='Catchiness Score: 8/10'>
Catchiness extracted: 8
Checking for 'Subreddit: r/' in text: False
Checking for 'arxiv.org' in text: False
Checking for 'RSS Feed:' in text: True
🔵 IDENTIFIED AS OTHER/RSS
RSS Feed: MIT Technology Review
Flags - Reddit: False, Research: False, Other: True
🔵 Base color set to DARK BLUE (Other/RSS)
Final color after brightness adjustment (catchiness=8.0): #8e8ecb

=== DocumentChunk #2 Debug ===
Text preview (first 200 chars): RSS Feed: The AI Blog



                Title: A conversation with Kevin Scott: What’s next in AI
                Publ
Catchiness match: <re.Match object; span=(358, 380), match='Catchiness Score: 5/10'>
Catchiness extracted: 5
Checking for 'Subreddit: r/' in text: False
Checking for 'arxiv.org' in t